In [9]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy3.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity.pkl')

In [2]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')

In [3]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        """
            K: 잠재요인(latent factor)의 수
            alpha: 학습률
            beta: 정규화 계수
            iterations: SGD의 계산을 할 때의 반복 횟수
            verbose: SGD의 중간 학습과정 출력 여부
        """
        self.R = np.array(ratings)
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    def rmse(self):
        """
            현재의 P행렬과 Q행렬을 가지고 Root Mean Squared Error(RMSE)를 계산
        """
        xs, ys = self.R.nonzero()
        # R에서 평점이 있는 (0이 아닌) 요소의 인덱스를 저장
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            # 평점이 존재하는 요소 (사용자 x, 아이템 y) 각각에 대해 아래 코드를 실행
            prediction = self.get_prediction(x, y)
            # 사용자 x, 아이템 y에 대한 평점 예측치를 계산
            self.predictions.append(prediction)
            # 예측값을 리스트에 추가
            self.errors.append(self.R[x, y] - prediction)
            # 실제값과 예측값의 차이를 계산해서 오차값 리스트에 추가
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        # errors를 사용해서 RMSE를 계산
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        """
            정해진 횟수만큼 반복하며 P, Q, bu, bd 값을 업데이트하는 함수
        """
        # Initializing user-feature and movie-feature matrix
        # P, Q 행렬을 임의의 값으로 채움
        # 평균이 0, 표준편차가 10/K인 정규분포를 갖는 난수로 초기화
        self.P = np.random.normal(scale=10./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=10./self.K, size=(self.num_items, self.K))
        
        # Initializing the bias terms
        # 사용자 평가 경향(bu)를 0으로 초기화 / 크기는 사용자 수 (num_users)
        # 아이템 평가 경향(bd)를 0으로 초기화 / 크기는 아이템 수 (num_items)
        # 전체 평균 b를 구해서 저장
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        # List of training samples
        # 평점 행렬에서 값이 존재하는 요소의 인덱스들을 가져옴
        # SGD를 적용할 대상 (평점이 있는 요소의 인덱스와 평점)을 리스트로 저장
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        
        # Stochastic gardient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            # samples를 임의로 섞음 => 어디서 시작하는지에 따라 수렴의 속도가 달라짐
            # 매 반복마다 다양한 시작점에서 시작하기!
            self.sgd()
            # sgd를 실행 / P, Q, bu, bd가 업데이트 됨
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("반복: {0}; Train RMSE = {1:0.4f}".format(i+1, rmse))
        return training_process
    
    def get_prediction (self, i, j):
        """
            Rating prediction for user i and item j
            사용자 i의 아이템 j에 대한 평점을 예측하는 함수
        """
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
        
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

    def sgd(self):
        """
            Stochastic gardient descent to get optimized P and Q matrix
        """
        for i, j, r in self.samples:
            # samples에 있는 각 사용자-아이템-평점 세트에 대해서 SGD를 적용
            prediction = self.get_prediction(i, j)
            # 사용자 i, 아이템 j에 대한 평점 예측치를 구함
            e = (r - prediction)
            # 실제 평점 r과 비교해서 오차를 구함
            
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
            # 사용자 평가 경향, 아이템 평가 경향 업데이트
            
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

In [4]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')
R_temp = view_matrix.fillna(0)
mf = MF(R_temp, K=40, alpha=0.001, beta=0.02, iterations=50, verbose=True)
# 인자 40개 / 반복 50회 / 학습률 0.001 / 정규화 계수 0.02로 모델을 학습합니다.
mf.train()
print(' ')

반복: 10; Train RMSE = 0.9267
반복: 20; Train RMSE = 0.8650
반복: 30; Train RMSE = 0.8250
반복: 40; Train RMSE = 0.7942
반복: 50; Train RMSE = 0.7676
 


In [5]:
print(mf.full_prediction())
# 전체 user-category의 예측값들입니다.

[[1.25038222 1.94136276 1.28719333 ... 2.18324927 1.96415727 1.61322725]
 [2.39629954 2.43460353 2.39244178 ... 2.67371018 1.74219861 2.05420801]
 [1.09856914 2.02879775 2.131331   ... 2.07444415 1.806203   1.85381593]
 ...
 [1.5151644  1.55432719 1.41004184 ... 1.36146968 1.11651943 1.1033737 ]
 [1.63102761 1.79417059 1.3037111  ... 1.21445968 1.16224693 0.96911566]
 [1.14672547 1.98366464 2.09009108 ... 1.07820989 1.40151173 1.45951949]]


In [6]:
testers = [
    {'user_id': 90001,'gender': '남성','age': 17}, {'user_id': 90081,'gender': '남성','age': 22}, {'user_id': 90161,'gender': '남성','age': 32}, {'user_id': 90221,'gender': '남성','age': 42}, {'user_id': 90281,'gender': '남성','age': 52}, {'user_id': 90341,'gender': '남성','age': 62}, {'user_id': 90401,'gender': '남성','age': 72},
    {'user_id': 90501,'gender': '여성','age': 12}, {'user_id': 90521,'gender': '여성','age': 22}, {'user_id': 90541,'gender': '여성','age': 32}, {'user_id': 90561,'gender': '여성','age': 42}, {'user_id': 90581,'gender': '여성','age': 52}, {'user_id': 90601,'gender': '여성','age': 62}, {'user_id': 90621,'gender': '여성','age': 72}
]

# 각각의 그룹에서 한명씩 유저를 데려오고... 그 유저의 추천 카테고리를 출력해봤습니다.
# 그룹화가 아니라 개인화 추천입니다!!! 같은 그룹이어도 다른 추천 결과가 나옵니다.
for tester in testers:
    print(tester['gender'], tester['age'])
    temp_data = dict()
    
    # 전체 카테고리에 대해 예측값을 계산합니다.
    for category in R_temp:
        temp_data[category] = mf.get_one_prediction(tester['user_id'], category)

    # 예측값을 기준으로 정렬합니다.
    recomm_df = pd.DataFrame.from_dict([temp_data]).T.reset_index().rename(columns={'index': 'category', 0: 'pred'})
    temp_recomm_df = recomm_df.sort_values(by='pred', ascending=False)
    temp_recomm_df['category'] = temp_recomm_df['category'].map(lambda x: category_dict[x])
    print(temp_recomm_df)

남성 17
     category      pred
140       SSD  2.672253
224        액자  2.644929
150     기타 게임  2.582096
385  기타 낚시 용품  2.580722
147       Wii  2.509907
..        ...       ...
231        접착  0.943906
282      안마의자  0.943003
236    관상어 사료  0.911671
131       마우스  0.892826
389       볼링공  0.783456

[423 rows x 2 columns]
남성 22
     category      pred
22        중지갑  2.984827
330  기타 수영 용품  2.978416
385  기타 낚시 용품  2.920209
313     유아 가구  2.847698
400      요가매트  2.844886
..        ...       ...
278       화장지  1.038245
290  기타 청소 용품  0.919937
245     세차 용품  0.899447
232     기타 공구  0.845799
253    기독교 용품  0.832020

[423 rows x 2 columns]
남성 32
     category      pred
144   플레이스테이션  3.504973
305  스킨/바디 용품  3.453676
420     기타 문구  3.255225
101     샴푸/린스  3.188699
155       망원경  3.138476
..        ...       ...
53      안경 소품  1.206423
109     남성 향수  1.194390
368     무술 용품  1.139734
291   구강위생 용품  1.078607
233     퍼즐/블록  1.013104

[423 rows x 2 columns]
남성 42
      category      pred
51       남성 양말 

In [ ]:
# 아래는 테스트중인 코드입니다. (svd 알고리즘) / https://www.fun-coding.org/recommend_basic6.html

In [50]:
view_matrix = view_matrix.fillna(0)
matrix_values = view_matrix.values
user_view_mean = np.mean(matrix_values, axis=1)

matrix_user_mean = matrix_values - user_view_mean.reshape(-1, 1)

In [51]:
from scipy.sparse.linalg import svds
user, sigma, vt = svds(matrix_user_mean, k = 12)

In [36]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(1140, 12)
(12,)
(12, 423)


In [37]:
sigma = np.diag(sigma)
print(sigma.shape)

(12, 12)


In [39]:
svd_user_prediction = np.dot(np.dot(U, sigma), Vt) + user_view_mean.reshape(-1, 1)
print(svd_user_prediction)

[[ 0.08840774  0.07475646  0.0300195  ...  1.44256488 -0.25870889
  -0.08365296]
 [ 0.12165599  0.0848846   0.11925023 ...  2.23502453 -0.43544207
  -0.15932123]
 [ 0.10616207  0.0816516   0.05085761 ...  1.28134168 -0.20772028
  -0.05917162]
 ...
 [-0.01167547 -0.01450485 -0.09076771 ...  0.49524846  0.50887437
   0.30460415]
 [-0.01167547 -0.01450485 -0.09076771 ...  0.49524846  0.50887437
   0.30460415]
 [-0.01167547 -0.01450485 -0.09076771 ...  0.49524846  0.50887437
   0.30460415]]


In [49]:
for i in range(0, 1140, 20):
    print(i, svd_user_prediction[i].argmax())

user      90601
gender        F
age          20
Name: 0, dtype: object
0 408
user      90541
gender        F
age          10
Name: 20, dtype: object
20 343
user      90841
gender        F
age          60
Name: 40, dtype: object
40 16
user      90161
gender        M
age          30
Name: 60, dtype: object
60 118
user      90381
gender        M
age          60
Name: 80, dtype: object
80 16
user      90861
gender        F
age          60
Name: 100, dtype: object
100 118
user      90501
gender        F
age          10
Name: 120, dtype: object
120 418
user      90581
gender        F
age          20
Name: 140, dtype: object
140 343
user      90241
gender        M
age          40
Name: 160, dtype: object
160 118
user      90901
gender        F
age          70
Name: 180, dtype: object
180 418
user      90921
gender        F
age          70
Name: 200, dtype: object
200 313
user      90441
gender        M
age          70
Name: 220, dtype: object
220 16
user      90121
gender        M
age        